# Generate tables for analysis

In [ ]:
%load_ext autoreload
%autoreload 2

from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import utils

pd.set_option('display.max_columns', 150)
pd.options.display.float_format = '{:,.5f}'.format
sns.set_theme(style="darkgrid")

# --- Constants ---
LARGE_CORP_MIN_SALES = 50
MED_CORP_MIN_SALES = 10
# -----------------

def format_thousands(value, tick_number):
    if value >= 1000:
        value = int(round(value / 1000, 0))
        return f"{value}K"
    else:
        return int(value)
    
def format_hundreds(value, tick_number):
    if value >= 1000:
        value = float(round(value / 1000, 1))
        return f"{value}K"
    else:
        return int(value)

def format_percent(value, tick_number):
    return f"{int(value)}%"

def format_millions(value, tick_number):
    return f"${int(value/1000000)}M"

# classify grantee/grantor by corp size
def classify_corp(x):
    if x > LARGE_CORP_MIN_SALES:
        return 3
    elif x > MED_CORP_MIN_SALES:
        return 2
    else:
        return 1
    
DATA_PATH = 'output/'

digest_atl = pd.read_csv("fcs_digest_atl.csv")
sales_atl = pd.read_csv("fcs_sales_atl.csv")

## Merge geodata with parcels and sales

---

## Figure 1. Total SFH Sales in Fulton County and Atlanta

In [ ]:
def calculate_counts(df, label, group_col='Sale Year'):
    return df.groupby(group_col)['PARID'].count().reset_index(name=label)

dfs = []
df_label = {
    'Total': sales,
    'Total Fulton Excl Atl': sales[sales["neighborhood"].isna()],
    'Valid Fulton Excl Atl': valid_sales[valid_sales["neighborhood"].isna()],
    'Invalid Fulton Excl Atl': invalid_sales[invalid_sales["neighborhood"].isna()],
    'Total Atlanta': sales_atl,
    'Valid Atlanta': valid_atl,
    'Invalid Atlanta': invalid_atl
}
for label, df in df_label.items():
    dfs.append(calculate_counts(df, label))

total_sales = reduce(lambda left, right: pd.merge(left, right, on='Sale Year'), dfs)

pct_cols = [
    "Valid Fulton Excl Atl",
    "Invalid Fulton Excl Atl",
    "Valid Atlanta",
    "Invalid Atlanta"
]

pcts = total_sales[pct_cols].divide(total_sales["Total"], axis=0).add_suffix(' %')

total_sales = pd.concat([total_sales, pcts], axis=1)
total_sales

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6), gridspec_kw={'wspace': .22})

utils.area_plot(
    total_sales,
    "Sale Year",
    ["Total", "Total Fulton Excl Atl", "Total Atlanta"],
    ["Total", "Total Fulton (excl. ATL)", "Total Atlanta"],
    "Sale Year",
    "Total Transactions",
    title="SF Transactions in Fulton County, GA",
    ax=ax[0],
    format_func=format_thousands,
    format_tuple=(0, 1),
    legend={"loc": "upper left", "bbox_to_anchor": (0, 0.97)}
)

utils.stacked_plot(
    total_sales,
    "Sale Year",
    ["Valid Fulton Excl Atl %", "Valid Atlanta %", "Invalid Fulton Excl Atl %", "Invalid Atlanta %"],
    ["Typical Fulton (excl. ATL)", "Typical Atlanta", "Atypical Fulton (excl. ATL)", "Atypical Atlanta"],
    "Sale Year",
    y_label="Proportion of Total Transactions",
    title="Proportion of SF Transactions in Fulton County, GA",
    ax=ax[1],
    legend={"loc": "lower right"},
    opacity=.85
)

ax[1].axhline(y=1, color='red', linestyle='--', alpha=.35)

## Figure 2. Total Sales by Corporates and Table of Invalid Sales

In [ ]:
# want total trans in Atlanta; total and invalid broken down by corp purchases and sales
dfs = []
df_label = {
    'Total Corp Trans Atl': sales_atl[(sales_atl['GRANTEE_corp_flag'] == 1) | (sales_atl['GRANTOR_corp_flag'] == 1)],
    'Total Corp Purchases Atl': sales_atl[sales_atl['GRANTEE_corp_flag'] == 1],
    'Total Corp Sales Atl': sales_atl[sales_atl['GRANTOR_corp_flag'] == 1],
    'Invalid Corp Purchases Atl': invalid_atl[invalid_atl['GRANTEE_corp_flag'] == 1],
    'Invalid Corp Sales Atl': invalid_atl[invalid_atl['GRANTOR_corp_flag'] == 1],
    'Valid Corp Purchases Atl': valid_atl[valid_atl['GRANTEE_corp_flag'] == 1],
    'Valid Corp Sales Atl': valid_atl[valid_atl['GRANTOR_corp_flag'] == 1]
}

for label, df in df_label.items():
    dfs.append(calculate_counts(df, label))

atlanta_breakdown = reduce(lambda left, right: pd.merge(left, right, on='Sale Year'), dfs)
total_sales = pd.merge(total_sales, atlanta_breakdown, on='Sale Year')
total_sales["Corp Trans Sum Atl"] = total_sales["Total Corp Purchases Atl"] + total_sales["Total Corp Sales Atl"]

pct_cols = [label for label in df_label.keys() if label != "Total Corp Trans Atl"]
pcts = total_sales[pct_cols].divide(total_sales["Corp Trans Sum Atl"], axis=0).add_suffix(' %')

pcts_of_corp_trans = total_sales[pct_cols].divide(total_sales["Corp Trans Sum Atl"], axis=0).add_suffix(' % Corp Trans')

total_sales = pd.concat([total_sales, pcts, pcts_of_corp_trans], axis=1)
total_sales

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(14, 14), gridspec_kw={'wspace': .22, 'hspace': .3})

for i, subset in enumerate(["Total", "Invalid"]):
    name = "Total"
    if i == 1:
        loc="upper right"
    else:
        loc="upper left"
    if subset == "Invalid":
        name = "Atypical"
    col_labels = ["Total", f"{subset} Corp Purchases Atl", f"{subset} Corp Sales Atl"]
    utils.area_plot(
        total_sales,
        "Sale Year",
        [f"{subset} Atlanta", f"{subset} Corp Purchases Atl", f"{subset} Corp Sales Atl"],
        [x.replace("Invalid", "Atypical") for x in col_labels],
        title=f"{name} SF Transcations in Atlanta, GA",
        x_label="Sale Year",
        y_label="Total Transactions",
        ax=ax[i][0],
        legend={"loc": loc},
        format_func=format_hundreds,
        format_tuple=(0, 1),
    )
    
    stacked_cols = [f"{subset} Corp Purchases Atl %", f"{subset} Corp Sales Atl %"]
    trans_level = " Corporate"
    if subset == "Invalid":
        stacked_cols = [
            "Invalid Corp Purchases Atl % Corp Trans",
            "Invalid Corp Sales Atl % Corp Trans",
            "Valid Corp Purchases Atl % Corp Trans",
            "Valid Corp Sales Atl % Corp Trans"
        ]
        trans_level = " Corporate"
    col_labels = [x.replace("Atl % Corp Trans", "").replace("Atl %", "").replace("Total Corp", "Corporate").replace("Valid", "Typical").replace("Invalid", "Atypical") for x in stacked_cols]
    if subset == "Invalid":
        col_labels = [x.replace(" Corp", "").replace("Valid", "Typical").replace("Invalid", "Atypical") for x in col_labels]


    utils.stacked_plot(
        total_sales,
        "Sale Year",
        stacked_cols,
        col_labels,
        title=f"Proportion of{trans_level} SF Transcations in Atlanta, GA",
        y_label=f"Proportion of{trans_level} Transactions",
        x_label="Sale Year",
        ax=ax[i][1],
        opacity=.65,
        legend={"loc": "lower right"}
    )
    
    ax[i][1].axhline(y=1, color='red', linestyle='--', alpha=.35)

## Figure 3. Sales by Transaction Scale
Note: switch with Figure 2 (e.g. this figure should come before previous) for paper

In [ ]:
# Note includes valid sales and purchases from govt, bank
sale_scale = pd.read_csv("output/sale_scale.csv")
sale_scale["TAXYR"] = (sale_scale["TAXYR"].astype(int) - 1)
sale_scale = sale_scale.rename(columns={"TAXYR": "Sale Year"})

In [ ]:
# Want to join sale_scale to sale so we have scale of both GRANTEE and GRANTOR
sales = sales.merge(
    sale_scale[["Sale Year", "entity_addr", "total_trans_fulton"]],
    left_on=["Sale Year", "GRANTEE_match_addr"], # GRANTEE scale
    right_on=["Sale Year", "entity_addr"],
    how="left"
).rename(columns={"total_trans_fulton": "Buyer Transactions Fulton"}).drop(
    columns=["entity_addr"]
).merge(
    sale_scale[["Sale Year", "entity_addr", "total_trans_fulton"]],
    left_on=["Sale Year", "GRANTOR_match_addr"], # GRANTOR scale
    right_on=["Sale Year", "entity_addr"],
    how="left"
).rename(columns={"total_trans_fulton": "Seller Transactions Fulton"}).drop(
    columns=["entity_addr"]
)

for col in ["Buyer Transactions Fulton", "Seller Transactions Fulton"]:
    sales[col] = sales[col].fillna(0).astype(int)

In [ ]:
sales.head(5)[["Sale Year", "GRANTEE_match_addr", "GRANTOR_match_addr", "Buyer Transactions Fulton", "Seller Transactions Fulton"]]

In [ ]:
sales["Buyer Scale"] = sales[
    sales["GRANTEE_corp_flag"] == 1
]["Buyer Transactions Fulton"].apply(classify_corp)

sales["Seller Scale"] = sales[
    sales["GRANTOR_corp_flag"] == 1
]["Seller Transactions Fulton"].apply(classify_corp)

for col in ["Buyer Scale", "Seller Scale"]:
    sales[col] = sales[col].fillna(0).astype(int)

sales_atl = sales[sales["neighborhood"].notna()]
# Add scale metrics to total sales table; e.g. count of sales by scale
# so we want to sum total sales, total valid sales where scale is 1, 2, 3 for each year
dfs = [total_sales] + []

scales = {
    0: "Individual",
    1: "Small Corporate",
    2: "Medium Corporate",
    3: "Large Corporate",
}

names = {
    "Seller": "Sales",
    "Buyer": "Purchases"
}

for trans_type in ["Buyer", "Seller"]:
    for i in scales.keys():
        df = calculate_counts(sales_atl[sales_atl[f"{trans_type} Scale"] == i], f"{scales[i]} {names[trans_type]} Atlanta")
        dfs.append(df)
        
total_sales = reduce(lambda left, right: pd.merge(left, right, on='Sale Year'), dfs)
total_sales

In [ ]:
pct_cols = [
    "Individual Purchases Atlanta",
    "Small Corporate Purchases Atlanta",
    "Medium Corporate Purchases Atlanta",
    "Large Corporate Purchases Atlanta",
    "Individual Sales Atlanta",
    "Small Corporate Sales Atlanta",
    "Medium Corporate Sales Atlanta",
    "Large Corporate Sales Atlanta"
]

pct_by_type = total_sales[pct_cols].divide(total_sales["Total Atlanta"], axis=0).add_suffix(' %')

total_sales = pd.concat([total_sales, pct_by_type], axis=1)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 12), gridspec_kw={'wspace': .22, 'hspace': .3})

for i, type in enumerate(["Purchases", "Sales"]):
    utils.area_plot(
        df=total_sales,
        x="Sale Year",
        cols=["Total Atlanta"] + [f"{scales[i]} {type} Atlanta" for i in scales.keys()],
        labels=[f"Total {type}"] + [f"{scales[i]} {type}".replace("Corporate", "Corp") for i in scales.keys()],
        x_label="Sale Year",
        y_label=f"Total {type}",
        title=f"SF {type} in Atlanta, GA",
        ax=ax[i][0],
        format_func=format_thousands,
        format_tuple=(0, 1),
        legend={"loc": "upper left"}
    )
    
    utils.stacked_plot(
        df=total_sales,
        x="Sale Year",
        cols=[f"{scales[i]} {type} Atlanta %" for i in scales.keys()],
        labels=[f"{scales[i]} {type}".replace("Corporate", "Corp") for i in scales.keys()],
        x_label="Sale Year",
        y_label=f"Proportion of {type}",
        title=f"Proportion of SF {type} in Atlanta, GA",
        ax=ax[i][1],
        opacity=.65,
        legend={"loc": "lower right"}
    )
    
    ax[i][1].axhline(y=1, color='red', linestyle='--', alpha=.35)

## Figure 4. Ownership of SFH Rentals

In [ ]:
owner_scale = pd.read_csv("output/owner_scale.csv")
owner_scale.head(2)

In [ ]:
digest = digest.merge(
    owner_scale[["TAXYR", "owner_addr", "count_owned_fulton_yr"]],
    on=["TAXYR", "owner_addr"],
    how="left"
)

digest["owner scale"] = digest[digest["own_corp_flag"] == 1]["count_owned_fulton_yr"].apply(classify_corp)
digest["owner scale"] = digest["owner scale"].fillna(0).astype(int)

In [ ]:
rentals = digest[digest["rental_flag"] == 1]
rentals_atl = rentals[rentals["neighborhood"].notna()]

num_rentals_fulton = calculate_counts(rentals, "Rentals Fulton", group_col="TAXYR")
num_rentals_atl = calculate_counts(rentals_atl, "Rentals Atlanta", group_col="TAXYR")

dfs = [num_rentals_fulton] + [num_rentals_atl] + []

for scale in scales.keys():
    df = calculate_counts(rentals[rentals["owner scale"] == scale], f"{scales[scale]} Fulton", group_col="TAXYR")
    dfs.append(df)
    
    df = calculate_counts(rentals_atl[rentals_atl["owner scale"] == scale], f"{scales[scale]} Atlanta", group_col="TAXYR")
    dfs.append(df)

rental_summary = reduce(lambda left, right: pd.merge(left, right, on='TAXYR'), dfs)

In [ ]:
dfs = [rental_summary] + []

pct_cols_fulton = [
    "Individual Fulton",
    "Small Corporate Fulton",
    "Medium Corporate Fulton",
    "Large Corporate Fulton",
]
pct_cols_atl = [
    "Individual Atlanta",
    "Small Corporate Atlanta",
    "Medium Corporate Atlanta",
    "Large Corporate Atlanta"
]

pct_rental_type_fulton = rental_summary[pct_cols_fulton].divide(rental_summary["Rentals Fulton"], axis=0).add_suffix(' %')
pct_rental_type_atl = rental_summary[pct_cols_atl].divide(rental_summary["Rentals Atlanta"], axis=0).add_suffix(' %')

dfs = dfs + [pct_rental_type_fulton, pct_rental_type_atl]

rental_summary = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True), dfs)
rental_summary

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6), gridspec_kw={'wspace': .22, 'hspace': .3})

area = "Atlanta"

utils.area_plot(
    rental_summary,
    "TAXYR",
    [f"Rentals {area}", f"Individual {area}", f"Small Corporate {area}", f"Medium Corporate {area}", f"Large Corporate {area}"],
    [f"Total Rentals", f"Individual", f"Small Corporate", f"Medium Corporate", f"Large Corporate"],
    "Tax Year",
    "Total SFRs",
    f"SFRs by Ownership in {area}, GA",
    legend={"loc": "center left"},
    ax=ax[0],
    format_func=format_thousands,
    format_tuple=(0, 1)
)

utils.stacked_plot(
    rental_summary,
    "TAXYR",
    [f"Individual {area} %", f"Small Corporate {area} %", f"Medium Corporate {area} %", f"Large Corporate {area} %"],
    [f"Individual", f"Small Corporate", f"Medium Corporate", f"Large Corporate"],
    "Tax Year",
    y_label="Proportion of SFRs",
    title=f"Proportion of SFRs in {area} by Ownership",
    legend={"loc": "lower left"},
    ax=ax[1],
    opacity=.65
)

ax[1].axhline(y=1, color='red', linestyle='--', alpha=.35)

## Figure 5. Neighborhood characteristics

In [ ]:
nsa_geo.rename(columns={"NEIGHBORHO": "neighborhood"}, inplace=True)
nsa_geo = nsa_geo[['STATISTICA', 'neighborhood', 'geometry']]

nsa_stats.rename(columns={"Details": "neighborhood"}, inplace=True)
nsa_stats = nsa_stats[[
    "GEOID",
    "neighborhood",
    "Median household income 2021",
    "% Not Hispanic Black or African American alone 2021",
]]

nsa_geo = nsa_geo.merge(
    nsa_stats,
    on="neighborhood",
    how="left"
)

dropped_nbhds = [
    'East Lake, The Villages at East Lake',
    'Edgewood',
    'Candler Park, Druid Hills',
    'Kirkwood',
    'Lake Claire',
    'East Atlanta',
    'Airport'
]

nsa_geo = nsa_geo[~nsa_geo["neighborhood"].isin(dropped_nbhds)]

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True)
hues = [
    "Median household income 2021",
    "% Not Hispanic Black or African American alone 2021",
]
titles = [
    "Median Household Income (2021, ACS)",
    "Percent Non-Hispanic Black (2021, ACS)",
]
highlight = [
    "Castleberry Hill, Downtown",
    "Buckhead Heights, Lenox, Ridgedale Park"
    #"Kingswood, Mt. Paran/Northside, Mt. Paran Parkway, Randall Mill, West Paces Ferry/Northside, Whitewater Creek",
    #"Chastain Park, Tuxedo Park",
    #'Adair Park, Pittsburgh',
    #'Lakewood, Leila Valley, Norwood Manor, Rebel Valley Forest',
]

utils.map(
    nsa_geo,
    color=hues[0],
    title=titles[0],
    nbhd_df=nsa_geo,
    ax=axes[0],
    highlight=highlight,
    format_func=format_thousands
)

utils.map(
    nsa_geo,
    color=hues[1],
    title=titles[1],
    nbhd_df=nsa_geo,
    ax=axes[1],
    highlight=highlight,
    format_func=format_percent
)

## Figure 6. Sales per SFH Parcel, Proportion Valid SFH Sales by Neighborhood

In [ ]:
dfs = []

dfs.append(calculate_counts(sales_atl, "Total Sales", group_col="neighborhood"))
dfs.append(calculate_counts(invalid_atl, "Invalid Sales", group_col="neighborhood"))
dfs.append(calculate_counts(digest_atl[digest_atl["TAXYR"] == 2022], "Total Parcels 2022", group_col="neighborhood"))

atl_sales_summary = reduce(lambda left, right: pd.merge(left, right, on='neighborhood'), dfs)

atl_sales_summary["Sales Per Parcel"] = atl_sales_summary["Total Sales"].divide(atl_sales_summary["Total Parcels 2022"], axis=0)
atl_sales_summary["Percent Invalid Sales"] = atl_sales_summary["Invalid Sales"].divide(atl_sales_summary["Total Sales"], axis=0)
atl_sales_summary = atl_sales_summary.merge(nsa_geo[["neighborhood", "geometry"]], on="neighborhood", how="left")

atl_sales_summary.sort_values("Sales Per Parcel", ascending=False).head(3)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

atl_sales_summary = atl_sales_summary.fillna(0)
#outliers_1 = ["Georgia Tech, Marietta Street Artery", "Lindbergh/Morosgo"]
highlight1 = [
    'Carver Hills, Rockdale, Scotts Crossing, West Highlands',
    'Lindbergh/Morosgo',
    'Adair Park, Pittsburgh',
]
highlight2 = [
    "English Avenue",
    'Atlanta University Center, The Villages at Castleberry Hill',
    'Center Hill, Harvel Homes Community',
]
utils.map(
    atl_sales_summary,
    color="Sales Per Parcel",
    title="Sales per SF Parcel by Neighborhood",
    nbhd_df=nsa_geo,
    ax=ax[0],
    log=True,
    highlight=highlight1,
)

utils.map(
    atl_sales_summary,
    color="Percent Invalid Sales",
    title="Proportion Atypical SF Sales by Neighborhood",
    nbhd_df=nsa_geo,
    ax=ax[1],
    highlight=highlight2,
)

## Figure 7. Percent of Sales at each Transaction Scale by Neighborhood

In [ ]:
digest_atl_2022 = digest[(digest["TAXYR"] == 2022) & (digest["neighborhood"].notna())]

dfs = [calculate_counts(digest_atl_2022, "Total Parcels 2022", group_col="neighborhood")] + []

for scale in scales:
    df = calculate_counts(
        digest_atl_2022[digest_atl_2022["owner scale"] == scale],
        f"{scales[scale]}",
        group_col="neighborhood"
    )
    
    dfs.append(df)
    
ownership_2022 = reduce(lambda left, right: pd.merge(left, right, on="neighborhood", how="left"), dfs)
ownership_2022 = ownership_2022.fillna(0)

pct_cols = [f"{scales[i]}" for i in scales.keys()]
pct_by_owner = ownership_2022[pct_cols].divide(ownership_2022["Total Parcels 2022"], axis=0).add_suffix(' %')
ownership_2022 = pd.concat([ownership_2022, pct_by_owner], axis=1)

ownership_2022 = ownership_2022.merge(
    nsa_geo[["neighborhood", "geometry"]],
    on="neighborhood",
    how="left"
)

ownership_2022.sort_values(by="Large Corporate %", ascending=False).head(3)

In [ ]:
# Proportion of parcels owned by individuals
digest_atl_2022[digest_atl_2022["own_corp_flag"] != 1]["PARID"].count() / digest_atl_2022["PARID"].count()

In [ ]:
# TODO: make 0 grayed out (could just plot a map underneath)
fig, ax = plt.subplots(2, 2, figsize=(12, 12), gridspec_kw={'wspace': .22, 'hspace': .3})

highlight = [
    [
        "English Avenue",
        'Lakewood, Leila Valley, Norwood Manor, Rebel Valley Forest',
    ],
    [
        'Atlanta University Center, The Villages at Castleberry Hill',
        'Lakewood, Leila Valley, Norwood Manor, Rebel Valley Forest',
    ],
    [
        "English Avenue",
        "Mechanicsville",
    ],
    [
        "South River Gardens",
         'Baker Hills, Bakers Ferry, Boulder Park, Fairburn Road/Wisteria Lane, Ridgecrest Forest, Wildwood (NPU-H), Wilson Mill Meadows, Wisteria Gardens',
    ]
]

log=False
for i, scale in enumerate(scales.values()):
    # neighborhoods we already excluded
    if i in [1, 2, 3]:
        log=True
    else:
        log=False
    utils.map(
        ownership_2022[~ownership_2022["neighborhood"].str.contains("Downtown")],
        color=f"{scale} %",
        title=f"Proportion Owned by {scale}s, 2022",
        nbhd_df=nsa_geo,
        ax=ax.flatten()[i],
        log=log,
        highlight=highlight[i],
    )

## Figure 8. Percent of Sales by Sale Type Map

In [ ]:
dfs = [] + [atl_sales_summary]

sale_types = ["corp_bought_ind", "corp_sold_ind", "ind_to_ind", "corp_to_corp"]
for sale in sale_types:
    dfs.append(pd.DataFrame(sales_atl[
        sales_atl[sale] == 1
    ].groupby("neighborhood")["PARID"].count()).rename(columns={"PARID": f"Total {sale}"}).reset_index())
    
atl_sales_summary = reduce(lambda left, right: pd.merge(left, right, on="neighborhood", how="left"), dfs)

atl_sales_summary = atl_sales_summary.fillna(0)
atl_sales_summary['Percent corp_bought_ind'] = atl_sales_summary['Total corp_bought_ind'] / atl_sales_summary['Total Sales']
atl_sales_summary['Percent corp_sold_ind'] = atl_sales_summary['Total corp_sold_ind'] / atl_sales_summary['Total Sales']
atl_sales_summary['Percent ind_to_ind'] = atl_sales_summary['Total ind_to_ind'] / atl_sales_summary['Total Sales']
atl_sales_summary['Percent corp_to_corp'] = atl_sales_summary['Total corp_to_corp'] / atl_sales_summary['Total Sales']
atl_sales_summary.sort_values(by="Percent corp_bought_ind", ascending=False).head(5)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 12), gridspec_kw={'wspace': .22, 'hspace': .3})

highlight1 = [
    "Hammond Park",
    'Bankhead Courts, Bankhead/Bolton, Carroll Heights, Fairburn Heights, Old Gordon',
]

highlight2 = [
    "Lindbergh/Morosgo",
    "Carver Hills, Rockdale, Scotts Crossing, West Highlands",
    "Fairburn Mays, Mays",
]

highlight3 = [
    'Collier Hills, Collier Hills North, Colonial Homes',
    'Grant Park, Oakland',
]

highlight4 = [
    "Almond Park, Carey Park",
    "Lakewood, Leila Valley, Norwood Manor, Rebel Valley Forest",
]

utils.map(
    atl_sales_summary,
    color="Percent corp_bought_ind",
    title="Proportion of Sales (Corporate Purchase from Individual)",
    nbhd_df=nsa_geo,
    ax=ax[0][0],
    log=True,
    highlight=highlight1
)
utils.map(
    atl_sales_summary,
    color="Percent corp_sold_ind",
    title="Proportion of Sales (Corporate Sale to Individuals)",
    nbhd_df=nsa_geo,
    ax=ax[0][1],
    log=True,
    highlight=highlight2
)
utils.map(
    atl_sales_summary,
    color="Percent ind_to_ind",
    title="Proportion of Sales (Individual to Individual)",
    nbhd_df=nsa_geo,
    ax=ax[1][0],
    highlight=highlight3
)
utils.map(
    atl_sales_summary,
    color="Percent corp_to_corp",
    title="Proportion of Sales (Corporate to Corporate)",
    nbhd_df=nsa_geo,
    ax=ax[1][1],
    highlight=highlight4
)

## Figure 9. Graph of Majority-Black vs Other Neighborhoods by Corporate Purchases and Sales

In [ ]:
# Separate neighborhoods into majority Black and other with mb_flag
mb_neighborhoods = nsa_geo[
    nsa_geo["% Not Hispanic Black or African American alone 2021"] >= 50
]["neighborhood"].unique()

sales["mb_flag"] = sales["neighborhood"].apply(lambda x: 1 if x in mb_neighborhoods else 0)
# Agg all sales on mb_flag, calculate total transcations, number where buyer is corp,
# seller is corp, buyer is corp and seller is corp, then calculate percent
total_sales = calculate_counts(sales, "total_sales", group_col=["mb_flag", "Sale Year"])

all_corp_trans = sales[(sales["GRANTEE_corp_flag"] == 1) | (sales["GRANTOR_corp_flag"] == 1)]
corp_trans = calculate_counts(all_corp_trans, "corp_trans", group_col=["mb_flag", "Sale Year"])

mb_summary = total_sales.merge(corp_trans, on=["mb_flag", "Sale Year"], how="left")
mb_summary["% corp"] = mb_summary["corp_trans"] / mb_summary["total_sales"]

In [ ]:
# TODO add in corp to corp / invalid
# TODO for each point, put absolute numbers

fig, ax = plt.subplots(1, 1, figsize=(6, 6))

utils.area_plot(
    mb_summary[mb_summary["mb_flag"] == 0],
    "Sale Year",
    ["% corp"],
    ["Non-Majority Black Neighborhood"],
    "Sale Year",
    title="Proportion of Transactions Involving a Corporate Entity",
    ax=ax,
    legend={"loc": "lower left"},
    ann="total_sales"
)

utils.area_plot(
    mb_summary[mb_summary["mb_flag"] == 1],
    "Sale Year",
    ["% corp"],
    ["Majority Black Neighborhood"],
    "Sale Year",
    "Proportion of Transactions",
    ax=ax,
    legend={"loc": "lower left"},
    ann="total_sales"
)

---
## Equity Loss Figures

## Calculation by Year
Drop outliers where sale_diff is too extreme (multi-parcel sales with properties outside Fulton most likely)

In [ ]:
# sale_diff = fmv_adj - price_adj
sales["sale_diff_adj"] = sales["fmv_adj"] - sales["price_adj"]
sales["sale_diff"] = sales["FAIR MARKET VALUE"] - sales["sales_price"]
original_sales = sales.copy(deep=True)

# Calculate the 99th percentile of sale_diff
top_threshold = sales["sale_diff_adj"].quantile(0.99)
bottom_threshold = sales["sale_diff_adj"].quantile(0.01)

# Filter out the rows where sale_diff is greater than the threshold
sales = sales[
    (sales["sale_diff_adj"] < top_threshold)
    & (sales["sale_diff_adj"] > bottom_threshold)
]
dropped_sales = original_sales[(original_sales["sale_diff_adj"] >= top_threshold) | (original_sales["sale_diff_adj"] <= bottom_threshold)]

In [ ]:
def agg(df, label, agg_col, group_col='Sale Year'):
    if isinstance(group_col, list):
        return df.groupby(by=group_col)[agg_col].sum().reset_index(name=label)
    else:
        return df.groupby(by=[group_col])[agg_col].sum().reset_index(name=label)
    
# sale loss: aggregate sale_diff where corp_sold_ind = 1
# positive indicates sold for below FMV, company contributed to neighborhood equity
dfs = []
atl_sales = sales[sales["neighborhood"].notna()]

dfs.append(agg(atl_sales[atl_sales["corp_sold_ind"] == 1], "sale_loss_adj", "sale_diff_adj", group_col=["TAXYR", "neighborhood"]))
dfs.append(agg(atl_sales[atl_sales["corp_bought_ind"] == 1], "purchase_loss_adj", "sale_diff_adj", group_col=["TAXYR", "neighborhood"]))

dfs.append(agg(atl_sales[atl_sales["corp_sold_ind"] == 1], "sale_loss", "sale_diff", group_col=["TAXYR", "neighborhood"]))
dfs.append(agg(atl_sales[atl_sales["corp_bought_ind"] == 1], "purchase_loss", "sale_diff", group_col=["TAXYR", "neighborhood"]))

dfs.append(agg(atl_sales[(atl_sales["corp_sold_ind"] == 1) & (atl_sales["GRANTOR_match"].str.contains("CHARIS"))], "sale_loss_adj_charis", "sale_diff_adj", group_col=["TAXYR", "neighborhood"]))
dfs.append(agg(atl_sales[(atl_sales["corp_bought_ind"] == 1) & (atl_sales["GRANTEE_match"].str.contains("CHARIS"))], "purchase_loss_adj_charis", "sale_diff_adj", group_col=["TAXYR", "neighborhood"]))

# rental loss: sum of aprtot_adj * .005 * 12 where rental_flag = 1 and own_corp_flag = 1
digest["rental_value_adj"] = digest["Aprtot_adj"] * .005 * 12
digest["rental_value"] = digest["Aprtot"] * .005 * 12

# ignore 2022 rental loss, since sale data ends at 2021
rental_loss_digest = digest[
    (digest["neighborhood"].notna())
    & (digest["rental_flag"] == 1)
    & (digest["own_corp_flag"] == 1)
    & (digest["TAXYR"] < 2022)
]
dfs.append(agg(rental_loss_digest, "rental_loss_adj", "rental_value_adj", group_col=["TAXYR", "neighborhood"]))
dfs.append(agg(rental_loss_digest[rental_loss_digest["Own1"].str.contains("CHARIS")], "rental_loss_adj_charis", "rental_value_adj", group_col=["TAXYR", "neighborhood"]))
dfs.append(agg(rental_loss_digest, "rental_loss", "rental_value", group_col=["TAXYR", "neighborhood"]))

equity_summary = reduce(lambda left, right: pd.merge(left, right, on=["TAXYR", "neighborhood"], how="left"), dfs)

# flip sign so negative is contributing, positive is loss
equity_summary["sale_loss_adj"] = -equity_summary["sale_loss_adj"]
equity_summary["total_loss_adj"] = equity_summary["sale_loss_adj"] + equity_summary["purchase_loss_adj"] + equity_summary["rental_loss_adj"]

equity_summary["sale_loss"] = -equity_summary["sale_loss"]
equity_summary["total_loss"] = equity_summary["sale_loss"] + equity_summary["purchase_loss"] + equity_summary["rental_loss"]

equity_summary = equity_summary.merge(
    nsa_geo,
    on="neighborhood",
    how="left"
)
equity_summary.head(3)

In [ ]:
equity_summary = equity_summary.fillna(0)

In [ ]:
household_data = pd.read_csv("data/atl_nsa_households.csv", skiprows=1).rename(columns={"Details": "neighborhood"})
household_data.head(2)

In [ ]:
# Agg on parcel level (e.g. sum sale_diff across all sales for each parcel), map

loss_per_parcel = atl_sales.groupby("PARID")["sale_diff_adj"].sum().reset_index(name="total_sale_diff_adj").sort_values(by="total_sale_diff_adj", ascending=False)

In [ ]:
loss_per_parcel = loss_per_parcel.merge(digest_atl[digest_atl["TAXYR"] == 2022], on="PARID", how="inner")
from shapely import wkt

loss_per_parcel['geometry'] = loss_per_parcel['geometry'].apply(wkt.loads)
loss_per_parcel = gpd.GeoDataFrame(loss_per_parcel, geometry='geometry')

In [ ]:
loss_per_parcel.explore(tiles="Stamen Toner", column="total_sale_diff_adj")

In [ ]:
loss_per_parcel.sort_values(by="total_sale_diff_adj", ascending=False).head(10)

## Total Calculation

In [ ]:
dfs = []
dfs.append(agg(equity_summary, "sale_loss_adj", "sale_loss_adj", group_col=["neighborhood"]))
dfs.append(agg(equity_summary, "purchase_loss_adj", "purchase_loss_adj", group_col=["neighborhood"]))
dfs.append(agg(equity_summary, "sale_loss_adj_charis", "sale_loss_adj_charis", group_col=["neighborhood"]))
dfs.append(agg(equity_summary, "purchase_loss_adj_charis", "purchase_loss_adj_charis", group_col=["neighborhood"]))
dfs.append(agg(equity_summary, "rental_loss_adj", "rental_loss_adj", group_col=["neighborhood"]))

dfs.append(agg(equity_summary, "sale_loss", "sale_loss", group_col=["neighborhood"]))
dfs.append(agg(equity_summary, "purchase_loss", "purchase_loss", group_col=["neighborhood"]))
dfs.append(agg(equity_summary, "rental_loss", "rental_loss", group_col=["neighborhood"]))

total_equity = reduce(lambda left, right: pd.merge(left, right, on="neighborhood", how="left"), dfs)
total_equity = total_equity.fillna(0)

total_equity["total_loss_adj"] = total_equity["sale_loss_adj"] + total_equity["purchase_loss_adj"] + total_equity["rental_loss_adj"]
total_equity["total_loss"] = total_equity["sale_loss"] + total_equity["purchase_loss"] + total_equity["rental_loss"]

total_equity = total_equity.merge(
    nsa_geo,
    on="neighborhood",
    how="left"
).merge(
    household_data[["neighborhood", "Average household size 2021", "# Total households 2021"]],
    on="neighborhood",
    how="left"
)

In [ ]:
total_equity[total_equity["rental_loss_adj"] > 0].sort_values("total_loss_adj", ascending=False).head(4)

In [ ]:
atl_sales[(atl_sales["GRANTOR_match"].str.contains("CHARIS")) & (atl_sales["neighborhood"] == "Thomasville Heights")].head(2)

In [ ]:
total_equity["total_loss_adj"].mean()

In [ ]:
f"{total_equity["total_loss_adj"].sum():,.0f}"

In [ ]:
digest[
    (digest["neighborhood"].str.contains("Kingswood"))
    & (digest["rental_flag"] == 1)
    & (digest["own_corp_flag"] == 1)
][["PARID", "Situs Adrno", "Situs Adrstr", "owner_addr", "Own1", "Aprtot_adj", "rental_value", "TAXYR"]].sort_values("rental_value", ascending=False).head(10)

In [ ]:
total_equity.sort_values(by="total_loss_adj", ascending=True).head(4)

## Figure 10. Purchase, Sale, and Rental Equity Loss Map

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 12), gridspec_kw={'wspace': .22, 'hspace': .3})

names = {
    "sale_loss_adj": "Sale",
    "purchase_loss_adj": "Purchase",
    "rental_loss_adj": "Rental",
    "total_loss_adj": "Total"
}

j = 0
i = 0

highlight = [
    [
        "Bush Moutain, Oakland City",
        "Adair Park, Pittsburgh",
        "Sylvan Hills"
    ],
    [
        "Morningside/Lenox Park",
        "Ormewood Park",
        "Grant Park, Oakland"
    ],
    [
        "Kingswood, Mt. Paran/Northside, Mt. Paran Parkway, Randall Mill, West Paces Ferry/Northside, Whitewater Creek",
        "Chastain Park, Tuxedo Park",
    ],
    [
        "Kingswood, Mt. Paran/Northside, Mt. Paran Parkway, Randall Mill, West Paces Ferry/Northside, Whitewater Creek",
        "Chastain Park, Tuxedo Park",
        "Ormewood Park",
        "Grant Park, Oakland",
        "Adair Park, Pittsburgh",
    ]
]

h = 0
for color in ["purchase_loss_adj", "sale_loss_adj", "rental_loss_adj", "total_loss_adj"]:
    utils.map(
        total_equity,
        color=color,
        title=f"{names[color]} Loss by Neighborhood",
        nbhd_df=nsa_geo,
        ax=ax[i][j],
        format_func=format_millions,
        highlight=highlight[h],
    )
    h += 1
    j += 1
    if j == 2:
        j = 0
        i += 1


## Figure 11. Equity Burden Map and Boxplot of Majority-Black vs Other Neighborhoods

In [ ]:
per_year_nsa = pd.read_csv("data/atl_nsa_by_year.csv").rename(
    columns={
        "tAggregateHHInc_e": "income_generated"
    }
)
per_year_nsa["NSA"] = per_year_nsa["NSA"].astype(str)
per_year_nsa["Year"] = per_year_nsa["Year"].astype(int)
equity_summary["GEOID"] = equity_summary["GEOID"].astype(str)
per_year_nsa["income_generated"] = per_year_nsa.groupby("NSA")["income_generated"].ffill()
per_year_nsa.head(2)

In [ ]:
equity_summary = equity_summary.merge(
    per_year_nsa,
    left_on=["GEOID", "TAXYR"],
    right_on=["NSA", "Year"],
)

In [ ]:
equity_summary.groupby("neighborhood")["total_loss"].sum().sort_values(ascending=False).head(3)

In [ ]:
equity_summary["equity_burden"] = equity_summary["total_loss"] / equity_summary["income_generated"]
equity_summary.sort_values(by="equity_burden", ascending=False).head(3)

In [ ]:
loss = equity_summary.groupby("neighborhood")["total_loss"].sum()
income = equity_summary.groupby("neighborhood")["income_generated"].sum()

equity_burden = pd.concat([loss, income], axis=1).reset_index()
equity_burden["equity_burden"] = equity_burden["total_loss"] / equity_burden["income_generated"]
equity_burden = equity_burden.merge(
    nsa_geo,
    on="neighborhood",
    how="left"
)
equity_burden["equity_burden"] = equity_burden["equity_burden"].fillna(0)
equity_burden.sort_values(by="equity_burden", ascending=False).head(3)

In [ ]:
total_equity.sort_values(by="total_loss", ascending=False).head(3)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

highlight = [
    'Chosewood Park, Englewood Manor',
    'Grove Park',
    'Browns Mill Park, Polar Rock, Swallow Circle/Baywood',
    'Bush Mountain, Oakland City',
]
utils.map(
    equity_burden,
    color="equity_burden",
    title="",
    nbhd_df=nsa_geo,
    ax=ax,
    highlight=highlight,
)

In [ ]:
equity_burden.sort_values("equity_burden", ascending=False).head(10)

In [ ]:
equity_burden["income_generated"].sum() - per_year_nsa[per_year_nsa["Year"] != 2012]["tAggregateHHInc_m"].sum()

In [ ]:
equity_summary.sort_values("equity_burden", ascending=False).head(1)

In [ ]:
total_equity.agg({"sale_loss": sum, "purchase_loss": sum, "rental_loss": sum, "total_loss": sum})

In [ ]:
total_equity["mb_flag"] = total_equity["neighborhood"].apply(lambda x: 1 if x in mb_neighborhoods else 0)

In [ ]:
total_equity.groupby("mb_flag")["total_loss_adj"].sum()

Average equity burden for mb and non-mb neighborhoods (sum mb generation, mb generation)

In [ ]:
equity_burden["mb_flag"] = equity_burden["neighborhood"].apply(lambda x: 1 if x in mb_neighborhoods else 0)

In [ ]:
equity_burden.groupby("mb_flag")["income_generated"].sum() * (digest["TAXYR"].nunique() - 1)

In [ ]:
equity_burden.groupby("mb_flag")["total_loss"].sum() / equity_burden.groupby("mb_flag")["income_generated"].sum() * 100

In [ ]:
equity_summary.sort_values("equity_burden", ascending=False).head(3)

In [ ]:
equity_burden[equity_burden["neighborhood"].str.contains("Bush")]

Change median to avg household income... neeed to change for BUsh mountain in section too

In [ ]:
# sum equity loss for each year by nbhd type
# sum household size * total households for each year by nbhd type
total_loss = equity_burden.groupby("mb_flag")["total_loss"].sum()
total_generation = equity_burden.groupby("mb_flag")["income_generated"].sum() * (digest["TAXYR"].nunique() - 1)

overall_pct = pd.concat([total_loss, total_generation], axis=1)
overall_pct["pct"] = overall_pct["total_loss"] / overall_pct["income_generated"] * 100
overall_pct

---
## Statistical Tests: Corporate Power

**Motivating question:** is the price paid (and sale_diff) different by sale type?

sale_diff = fmv_adj - price_adj

sale_type = corp to ind, ind to corp, etc.

**Model type**: Mixed Linear Model

**Model characteristics**
- Groups: neighborhoods and sale year (need to drop first year to avoid dummy trap)
- Treatment: type of sale (also need to avoid dummy trap, reference param in statsmodels library should do this)
- Dependent vars: sales price (log), sales_diff
- Covariates: 
    - Fixed effect: whether sale is valid, num beds, num baths, sqft living area, year built, heat
    - Random effect: sale year as level 1, neighborhoods nested within as level 2 - ADD TO MODEL

**Notes**
- Allow slopes and intercepts to vary?
- add in taxyr dummy, omitting first year (0): need to drop first year
- num bath, num bed, amenities, lot size, AC or not
- cluster standard errors at nbhd level

**Steps (do separately for each DV)**
- **Run model**
- **Residuals vs fitted plot (entire model):** verify there are no patterns, should be randomly distributed around zero
- **Residuals for each predictor variable:** verify no patterns
- **Distribution of residuals (QQ plot):** verify normal
- **Distribution of random effects (QQ plot):** verify normal
- **Variance inflation factors of IVs:** verify none are above 5-10
- **Correlation matrix of IVs:** verify all below 0.8
- **Degrees of Freedom:**
- **Confidence Intervals:** verify CIs don't include 0

In [ ]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
from statsmodels.stats.outliers_influence import variance_inflation_factor

Prepare data: cast all int/float, take log of sales price, create dummy column

In [ ]:
# Use heat, 4 = Central Air, 3 is forced air, 2 non central, 1 is none
# from understanding the PRC
sales_stat_vars = {
    "Sale Year": "int",
    "neighborhood": "string",
    "corp_sold_ind": "int",
    "corp_bought_ind": "int",
    "ind_to_ind": "int",
    "corp_to_corp": "int",
    "Saleval": "string",
    "sales_price": "float",
    "FAIR MARKET VALUE": "float",
    "price_adj": "float",
    "sale_diff": "float",
    "sqft_living": "float",
    "yr_built": "int",
    "beds": "int",
    "baths": "int",
    "acres": "float",
    "heat": "int",
    "D Effyr": "int",
    "Extwall": "int",
    "Style": "string",
    "Rmtot": "int",
    "D Grade": "string",
    "Bsmt": "int"
}
sale_type_names = [
    "ind_to_ind",
    "corp_to_corp",
    "corp_bought_ind",
    "corp_sold_ind",
]
sales_atl = sales[sales["neighborhood"].notna()]
sales_atl = sales_atl.merge(digest[
    ["TAXYR", "PARID", "yr_built", "sqft_living", "beds", "baths", "acres", "heat", "D Effyr", "Extwall", "Style", "Rmtot", "D Grade", "Bsmt"]
], on=["PARID", "TAXYR"], how="left")
sales_for_stat = sales_atl[sales_stat_vars.keys()].copy()
init_len = len(sales_for_stat)
sales_for_stat = sales_for_stat.dropna()
print(f"Dropped {init_len - len(sales_for_stat)} records due to missing values from digest merge")
sales_for_stat["sale_type"] = pd.from_dummies(sales_for_stat[sale_type_names])
sales_for_stat["valid_sale"] = sales_for_stat.apply(
    lambda x: x["Saleval"] == "0",
    axis=1
).astype(int)

sales_for_stat["sale_type"] = pd.from_dummies(sales_for_stat[sale_type_names])

for col, dtype in sales_stat_vars.items():
    sales_for_stat[col] = sales_for_stat[col].astype(dtype)

# natural log of sales price (adj for inflation)
sales_for_stat["price_adj_log"] = np.log(sales_for_stat["price_adj"])
sales_for_stat.rename(columns={"Sale Year": "sale_year"}, inplace=True)

# save data for validation
#sales_for_stat.to_csv("output/sales_for_stat.csv", index=False)

Data Check for Dist of sale_diff: fmv_adj - price_adj

In [ ]:
sales_for_stat["sale_diff"].describe()

Helper Functions

In [ ]:
def run_model(df, formula, groups, re_formula="1"):
    formula = formula
    model = smf.mixedlm(formula, df, groups=df[groups], re_formula=re_formula)
    return model.fit()

def verbose_output(result):
    print(result.summary())
    print(result.random_effects)
    print(result.cov_re)
    print(result.cov_params())
    
    # for name, coef in zip(mixed_lm_result.model.exog_names, mixed_lm_result.params):
    #     percent_change = (np.exp(coef) - 1) * 100
    #     print(f"{name}: {percent_change:.2f}%")
    
def model_residuals(result):
    fitted_vals = result.fittedvalues
    residuals = result.resid
    plt.scatter(fitted_vals, residuals)
    plt.xlabel('Fitted values')
    plt.ylabel('Residuals')
    plt.axhline(0, color='red', linestyle='--')
    plt.show()
    
def predictor_residuals(df, result, predictor):
    residuals = result.resid
    plt.scatter(df[predictor], residuals)
    plt.xlabel(f'{predictor}')
    plt.ylabel('Residuals')
    plt.axhline(0, color='red', linestyle='--')
    plt.show()
    
def dist_residuals(result):
    residuals = result.resid
    fig = sm.qqplot(residuals, stats.t, fit=True, line="45")
    plt.show()
    
def dist_random_effects(result):
    random_effects = result.random_effects
    # Flatten the array for plotting
    #re_flat = np.concatenate([re.flatten() for re in random_effects.values()])
    print(random_effects)
    fig = sm.qqplot(random_effects, stats.t, fit=True, line="45")
    plt.show()
    
def vif(df, predictors):
    pred = df[predictors]
    vif = pd.DataFrame()
    vif["VIF Factor"] = [variance_inflation_factor(pred.values, i) for i in range(pred.shape[1])]
    vif["features"] = pred.columns
    print(vif)
    
def corr_matrix(df, predictors):
    pred = df[predictors]
    corr_matrix = pred.corr()
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
    plt.show()

def run_diagnostics(df, result, predictors):
    model_residuals(result)
    
    for predictor in predictors:
        predictor_residuals(df, result, predictor)
        
    dist_residuals(result)
    #dist_random_effects(result)
    #dist_random_effects(result, predictors)
    vif(df, predictors)
    corr_matrix(df, predictors)

### Model - Sales Price

In [ ]:
formula = 'price_adj_log ~ C(sale_type, Treatment(reference="ind_to_ind")) + C(valid_sale) + beds + baths + sqft_living + yr_built + C(heat) + C(sale_year)'
result = run_model(df=sales_for_stat, formula=formula, groups="neighborhood")
verbose_output(result)

In [ ]:
sales_for_stat["price_adj_log"].describe()

In [ ]:
for name, coef in zip(result.model.exog_names, result.params):
    percent_change = (np.exp(coef) - 1) * 100
    print(f"{name}: {percent_change:.2f}%")

Diagnostics

In [ ]:
run_diagnostics(sales_for_stat, result, ["beds", "baths", "sqft_living", "yr_built", "heat"])

### Model - Sale Diff

In [ ]:
formula = 'sale_diff ~ sale_type + valid_sale + num_beds + num_baths + sqft_living + year_built + heat'
result = run_model(df=sales_for_stat, formula=formula, groups="neighborhood", re_formula="~sale_year")
verbose_output(result)

Diagnostics

In [ ]:
run_diagnostics(sales_for_stat, result, ["num_beds", "num_baths", "sqft_living", "year_built", "heat"])